<a href="https://colab.research.google.com/github/BRama10/Bayesian-ML-Notebooks-SampleWork/blob/main/Bayesian_Optimization_On_Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Displays runtime for a particular cell

!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 233 µs (started: 2022-07-29 00:47:53 +00:00)


#Ionosphere - Working with Bayesian Optimization using skopt


##Import dataset

In [ ]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv')
X, y = df.iloc[:, :-1], df.iloc[:, -1]
df

,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.1,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
0,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
1,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
2,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
3,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g
4,1,0,0.02337,-0.00592,-0.09924,-0.11949,-0.00763,-0.11824,0.14706,0.06637,...,-0.01535,-0.03240,0.09223,-0.07859,0.00732,0.00000,0.00000,-0.00039,0.12011,b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,1,0,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,...,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307,g
346,1,0,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,...,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712,g
347,1,0,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,...,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577,g
348,1,0,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,...,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243,g


time: 888 ms (started: 2022-07-29 00:47:57 +00:00)


##Pre-ML steps

In [ ]:
#Import Scikit-Optimize library.

!pip install scikit-optimize
import skopt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC

#print('skopt %s' % skopt.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 3.8 MB/s 
time: 5.95 s (started: 2022-07-29 00:48:11 +00:00)


In [ ]:
#Check original score

from numpy import mean,std

clf = SVC()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
m_scores = cross_val_score(clf, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
#print('Accuracy: %.3f (%.3f)' % (mean(m_scores), std(m_scores)))
mean(m_scores)

NameError: ignored

##Introduction to Minimizing Hyperparameters

In [ ]:
#Grid Search (Small amount of hyperparameters)

from sklearn.model_selection import GridSearchCV

params1 = {
    'C' : [0.1,1,2,3],
    'gamma' : [0.1,1,2,3,4],
    'degree' : [1,3,5],
    'kernel' : ['poly', 'rbf', 'sigmoid']
}

grid = GridSearchCV(SVC(), params1, cv = cv)

grid.fit(X,y)

print(grid.best_score_)
print(grid.best_params_)

0.9457142857142858
{'C': 3, 'degree': 1, 'gamma': 0.1, 'kernel': 'rbf'}
time: 1min 4s (started: 2022-07-28 15:35:45 +00:00)


In [ ]:
from skopt.space.space import Real, Categorical, Integer
from skopt.utils import use_named_args
from skopt import gp_minimize

params = list()
params.append(Real(1e-6, 100.0, 'log-uniform', name='C'))
params.append(Categorical(['linear', 'poly', 'rbf', 'sigmoid'], name='kernel'))
params.append(Integer(1, 5, name='degree'))
params.append(Real(1e-6, 100.0, 'log-uniform', name='gamma'))

In [ ]:
#Manual Tuning of Hyperparameters using skopt

@use_named_args(params)
def optimize_model(**params):
  clf = SVC()
  clf.set_params(**params)
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
  result = cross_val_score(clf, X, y, cv=cv, n_jobs=-1, scoring='accuracy')
  avg_result = mean(result)
  return 1.0 - avg_result

best_model = gp_minimize(optimize_model, params)

print('Best Accuracy: %.6f' % (1.0 - best_model.fun))
#Order = C, kernel, degree, gamma
print('Best Parameters: %s' % (best_model.x))


Best Accuracy: 0.952381
Best Parameters: [7.844235604183496, 'rbf', 1, 0.05434269355079624]
time: 2min 42s (started: 2022-07-27 23:59:35 +00:00)


In [ ]:
#Iteration 1 - Bayes
from skopt.space.space import Real, Categorical, Integer
from skopt.utils import use_named_args
from skopt import BayesSearchCV

params = dict()
params['C'] = (1e-6, 100.0, 'log-uniform')
params['gamma'] = (1e-6, 100.0, 'log-uniform')
params['degree'] = (1,5)
params['kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

search = BayesSearchCV(estimator=SVC(), search_spaces=params, n_jobs=-1, cv=cv)#, scoring = 'accuracy')

search.fit(X, y)

print(search.best_score_)
print(search.best_params_)

0.9257142857142857
OrderedDict([('C', 4.51096379491484e-06), ('degree', 2), ('gamma', 70.93273060027764), ('kernel', 'poly')])
time: 2min 13s (started: 2022-07-28 00:02:43 +00:00)


In [ ]:
#Iteration 2 - Bayes
from skopt.space.space import Real, Categorical, Integer
from skopt.utils import use_named_args
from skopt import BayesSearchCV

params = dict()
params['C'] = (1e-6, 100.0, 'log-uniform')
params['gamma'] = (1e-6, 100.0, 'log-uniform')
params['degree'] = (1,5)
params['kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

search = BayesSearchCV(estimator=SVC(), search_spaces=params, n_jobs=-1, cv=cv)#, scoring = 'accuracy')

search.fit(X, y)

print(search.best_score_)
print(search.best_params_)

0.9019047619047619
OrderedDict([('C', 0.1458787854160873), ('degree', 2), ('gamma', 2.2706908732449245), ('kernel', 'poly')])
time: 2min 12s (started: 2022-07-28 00:05:40 +00:00)


In [ ]:
#Iteration 3 - Bayes
from skopt.space.space import Real, Categorical, Integer
from skopt.utils import use_named_args
from skopt import BayesSearchCV

params = dict()
params['C'] = Real(1e-6, 100.0, 'log-uniform')
params['gamma'] = Real(1e-6, 100.0, 'log-uniform')
params['degree'] = Integer(1,5)
params['kernel'] = Categorical(['linear', 'poly', 'rbf', 'sigmoid'])

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

search = BayesSearchCV(estimator=SVC(), search_spaces=params, n_jobs=-1, cv=cv)#, scoring = 'accuracy')

search.fit(X, y)

print(search.best_score_)
print(search.best_params_)

0.9504761904761904
OrderedDict([('C', 3.8414527357369823), ('degree', 5), ('gamma', 0.10416742900674887), ('kernel', 'rbf')])


##Bayes Search vs GridSearch with large amounts of possible parameters

In [ ]:
#Iteration - Many Hyperparameter Values (GridSearch) (Unrealistic - with respect to degree)
from sklearn.model_selection import GridSearchCV

params1 = {
    'C' : [0.000001,0.000001,0.00001,0.0001,0.001,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
    'gamma' : [0.000001,0.000001,0.00001,0.0001,0.001,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
    'degree' : [0.000001,0.000001,0.00001,0.0001,0.001,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
    'kernel' : ['poly', 'rbf', 'sigmoid', 'linear']
}

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

grid = GridSearchCV(SVC(), params1, cv = cv)

grid.fit(X,y)

print(grid.best_score_)
print(grid.best_params_)

0.9447619047619048
{'C': 0.8, 'degree': 1e-06, 'gamma': 0.2, 'kernel': 'rbf'}
time: 39min 7s (started: 2022-07-28 12:52:46 +00:00)


In [ ]:
#Iteration - Many Hyperparameter Values (GridSearch) (Unrealistic - with respect to degree)
from sklearn.model_selection import GridSearchCV

params1 = {
    'C' : [0.0000001,0.000001,0.00001,0.0001,0.001,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
    'gamma' : [0.0000001,0.000001,0.00001,0.0001,0.001,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
    'degree' : [0.0000001,0.000001,0.00001,0.0001,0.001,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
    'kernel' : ['poly', 'rbf', 'sigmoid', 'linear']
}

grid = GridSearchCV(SVC(), params1, cv = cv)

grid.fit(X,y)

print(grid.best_score_)
print(grid.best_params_)

0.9447619047619048
{'C': 0.8, 'degree': 1e-07, 'gamma': 0.2, 'kernel': 'rbf'}
time: 43min 23s (started: 2022-07-28 01:05:41 +00:00)


In [ ]:
#Iteration - Many Hyperparameter Values (Bayes) (Unrealistic - with respect to degree)
from skopt.space.space import Real, Categorical, Integer
from skopt.utils import use_named_args
from skopt import BayesSearchCV

params = dict()
params['C'] = (1e-6, 100.0, 'log-uniform')
params['gamma'] = (1e-6, 100.0, 'log-uniform')
params['degree'] = (1e-6,100.0, 'log-uniform')
params['kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

search = BayesSearchCV(estimator=SVC(), search_spaces=params, n_jobs=-1, cv=cv)

search.fit(X, y)

print(search.best_score_)
print(search.best_params_)

0.9495238095238094
OrderedDict([('C', 1.751423170930387), ('degree', 0.02559216724871635), ('gamma', 0.34157708512609775), ('kernel', 'rbf')])
time: 2min (started: 2022-07-28 01:02:46 +00:00)


In [ ]:
#Iteration - Many Hyperparameter Values (GridSearch) (Realistic - with respect to degree)
from sklearn.model_selection import GridSearchCV

params1 = {
    'C' : [0.000001,0.000001,0.00001,0.0001,0.001,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
    'gamma' : [0.000001,0.000001,0.00001,0.0001,0.001,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
    'degree' : [1,2,3,4,5,6,7,8,9,10,11,12,13],
    'kernel' : ['poly', 'rbf', 'sigmoid', 'linear']
}

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

grid = GridSearchCV(SVC(), params1, cv = cv)

grid.fit(X,y)

print(grid.best_score_)
print(grid.best_params_)

0.9447619047619048
{'C': 0.8, 'degree': 1, 'gamma': 0.2, 'kernel': 'rbf'}
time: 40min (started: 2022-07-28 13:34:10 +00:00)


In [ ]:
#Iteration - Many Hyperparameter Values (Bayes) (Realistic - with respect to degree) Run 1
from skopt.space.space import Real, Categorical, Integer
from skopt.utils import use_named_args
from skopt import BayesSearchCV

params = dict()
params['C'] = Real(1e-6, 100.0, 'log-uniform')
params['gamma'] = Real(1e-6, 100.0, 'log-uniform')
params['degree'] = Integer(1,13, 'log-uniform')
params['kernel'] = Categorical(['linear', 'poly', 'rbf', 'sigmoid'])

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

search = BayesSearchCV(estimator=SVC(), search_spaces=params, n_jobs=-1, cv=cv)

search.fit(X, y)

print(search.best_score_)
print(search.best_params_)

0.92
OrderedDict([('C', 14.8309183283125), ('degree', 2), ('gamma', 0.11201207390489717), ('kernel', 'poly')])
time: 2min 9s (started: 2022-07-28 15:14:22 +00:00)


In [ ]:
#Iteration - Many Hyperparameter Values (Bayes) (Realistic - with respect to degree) Run 2
from skopt.space.space import Real, Categorical, Integer
from skopt.utils import use_named_args
from skopt import BayesSearchCV

params = dict()
params['C'] = Real(1e-6, 100.0, 'log-uniform')
params['gamma'] = Real(1e-6, 100.0, 'log-uniform')
params['degree'] = Integer(1,13, 'log-uniform')
params['kernel'] = Categorical(['linear', 'poly', 'rbf', 'sigmoid'])

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

search = BayesSearchCV(estimator=SVC(), search_spaces=params, n_jobs=-1, cv=cv)

search.fit(X, y)

print(search.best_score_)
print(search.best_params_)

0.9533333333333334
OrderedDict([('C', 7.764876756309389), ('degree', 7), ('gamma', 0.05988399800276573), ('kernel', 'rbf')])
time: 2min 9s (started: 2022-07-28 15:19:37 +00:00)
